In [1]:
import pandas as pd

In [4]:
df=pd.read_csv("C://Users//HP//Downloads//Participants_Data_analytics_olympiad_2023//uber_rides_data.xlsx - sample_train.csv")

In [5]:
df.head()

,ride_id,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


# shape of dataframe

In [6]:
df.shape

(200000, 8)

# check column data types

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ride_id            200000 non-null  int64  
 1   fare_amount        200000 non-null  float64
 2   pickup_datetime    200000 non-null  object 
 3   pickup_longitude   200000 non-null  float64
 4   pickup_latitude    200000 non-null  float64
 5   dropoff_longitude  199999 non-null  float64
 6   dropoff_latitude   199999 non-null  float64
 7   passenger_count    200000 non-null  int64  
dtypes: float64(5), int64(2), object(1)
memory usage: 12.2+ MB


# missing values in 'dropoff_longitude'

In [8]:
missing_values_count = df['dropoff_longitude'].isna().sum()

In [9]:
# missing_values_count = df['dropoff_longitude'].isnull().sum()

print("Number of missing values in 'dropoff_longitude' column:", missing_values_count)

Number of missing values in 'dropoff_longitude' column: 1


# convert 'pickup_datetime' to datetime datatype

In [10]:
pd.to_datetime(df['pickup_datetime'])


0        2015-05-07 19:52:06+00:00
1        2009-07-17 20:04:56+00:00
2        2009-08-24 21:45:00+00:00
3        2009-06-26 08:22:21+00:00
4        2014-08-28 17:47:00+00:00
                    ...           
199995   2012-10-28 10:49:00+00:00
199996   2014-03-14 01:09:00+00:00
199997   2009-06-29 00:42:00+00:00
199998   2015-05-20 14:56:25+00:00
199999   2010-05-15 04:08:00+00:00
Name: pickup_datetime, Length: 200000, dtype: datetime64[ns, UTC]

# remove null values

In [11]:
df.dropna(inplace=True)  #  remove rows with any missing values

# average fare amount

In [12]:
df_cleaned = df.dropna(subset=['fare_amount'])

# Calculate the average fare amount
average_fare = df_cleaned['fare_amount'].mean()

print("The average fare amount is:", average_fare)

The average fare amount is: 11.359891549457748


# median Haversine distance between pickup and dropoff locations

In [13]:
import numpy as np

def haversine(lat1, lon1, lat2, lon2):
    
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371  # Radius of the Earth in kilometers. Use 3959 for miles.
    
    return c * r


In [14]:

df['haversine_distance'] = df.apply(lambda row: haversine(row['pickup_latitude'], row['pickup_longitude'], row['dropoff_latitude'], row['dropoff_longitude']), axis=1)


In [16]:
median_distance = df['haversine_distance'].median()
print("The median Haversine distance between pickup and dropoff locations is:", median_distance, "kilometers")


The median Haversine distance between pickup and dropoff locations is: 2.1209923961833708 kilometers


# maximum Haversine distance between pickup and dropoff locations

In [17]:
maximum_distance = df['haversine_distance'].max()
print("The maximum Haversine distance between pickup and dropoff locations is:", maximum_distance, "kilometers")


The maximum Haversine distance between pickup and dropoff locations is: 16409.239135313164 kilometers


# number of rides with 0.0 Haversine distance between pickup and dropoff locations

In [18]:

rides_with_zero_distance = (df['haversine_distance'] == 0.0).sum()

print("The number of rides with 0.0 Haversine distance between pickup and dropoff locations is:", rides_with_zero_distance)


The number of rides with 0.0 Haversine distance between pickup and dropoff locations is: 5632


# mean 'fare_amount' for rides

In [19]:
mean_fare_for_zero_distance = df[df['haversine_distance'] == 0.0]['fare_amount'].mean()

print("The mean 'fare_amount' for rides with 0.0 Haversine distance is:", mean_fare_for_zero_distance)

The mean 'fare_amount' for rides with 0.0 Haversine distance is: 11.585317826704546


# maximum 'fare_amount'

In [20]:
maximum_fare_amount = df['fare_amount'].max()

print("The maximum 'fare_amount' for a ride is:", maximum_fare_amount)

The maximum 'fare_amount' for a ride is: 499.0


# Haversine distance for the costliest ride

In [21]:
#  row with the highest 'fare_amount'
costliest_ride = df[df['fare_amount'] == df['fare_amount'].max()]

# Calculate the Haversine distance for the costliest ride
haversine_distance_costliest_ride = haversine(
    costliest_ride['pickup_latitude'].values[0],
    costliest_ride['pickup_longitude'].values[0],
    costliest_ride['dropoff_latitude'].values[0],
    costliest_ride['dropoff_longitude'].values[0]
)

print("The Haversine distance for the costliest ride is:", haversine_distance_costliest_ride, "kilometers")


The Haversine distance for the costliest ride is: 0.0007899213191009994 kilometers


# recorded rides in the year 2014 

In [22]:
# Convert 'pickup_datetime' to a datetime object
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])

# Extract the year from 'pickup_datetime'
df['year'] = df['pickup_datetime'].dt.year

# Count the number of rides in the year 2014
rides_in_2014 = (df['year'] == 2014).sum()

print("The number of rides recorded in the year 2014 is:", rides_in_2014)

The number of rides recorded in the year 2014 is: 29968


# recorded rides in the first quarter of year 2014

In [23]:
#  filter for the first quarter of 2014
first_quarter_2014_filter = (df['pickup_datetime'] >= '2014-01-01') & (df['pickup_datetime'] <= '2014-03-31')

# filter to count the number of rides in the first quarter of 2014
rides_in_first_quarter_2014 = first_quarter_2014_filter.sum()

print("The number of rides recorded in the first quarter of 2014 is:", rides_in_first_quarter_2014)

The number of rides recorded in the first quarter of 2014 is: 7617


# maximum rides recorded day

In [24]:
# Extract the day of the week (0 = Monday, 6 = Sunday)
df['day_of_week'] = df['pickup_datetime'].dt.dayofweek

# Filter data for September 2010
september_2010_data = df[(df['pickup_datetime'].dt.year == 2010) & (df['pickup_datetime'].dt.month == 9)]

# Group by day of the week and count rides
rides_by_day_of_week = september_2010_data.groupby('day_of_week').size()

#  day with the maximum number of rides
max_rides_day = rides_by_day_of_week.idxmax()  # This gives the index (0-6) of the day

# Convert the index to the corresponding day name
day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
max_rides_day_name = day_names[max_rides_day]

print("The day of the week in September 2010 with the maximum rides recorded was:", max_rides_day_name)


The day of the week in September 2010 with the maximum rides recorded was: Thursday


# Model training and prediction

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [28]:
# Prepare the data
X = df[['passenger_count', 'haversine_distance', 'day_of_week']]
y = df['fare_amount']

In [29]:
# Split the data into training and testing sets (70-30 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [30]:
# Initialize and train different regression models
models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(),
    'Gradient Boosting': GradientBoostingRegressor()
}

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    adjusted_r2 = 1 - (1 - r2) * ((len(y_test) - 1) / (len(y_test) - len(X_test.columns) - 1))
    results[name] = adjusted_r2

# Determine the algorithm with the least adjusted R-squared
min_adjusted_r2_algorithm = min(results, key=results.get)
print("Algorithm with the least adjusted R-squared:", min_adjusted_r2_algorithm)

Algorithm with the least adjusted R-squared: Linear Regression
